In [1]:
if (!require('igraph')){ 
    install.packages('igraph')
}
library('igraph')
set.seed(666)

Loading required package: igraph
Warning message:
"package 'igraph' was built under R version 3.6.3"
Attaching package: 'igraph'

The following objects are masked from 'package:stats':

    decompose, spectrum

The following object is masked from 'package:base':

    union



In [2]:
fb_graph <- read.graph("facebook_combined.txt", format="edgelist", directed=FALSE)

In [3]:
create_core_node_graph <- function(id, graph){
    old_node_names <- c(id, neighbors(graph,id))
    core_node_g <- induced_subgraph(graph, old_node_names)
    #V(core_node_g)$name <- sort(old_node_names) # we do not need to hold old ids of vertices
    core_node_g$core_node_id <- id
    core_node_g$core_node_pos <- which(V(core_node_g)$name==id)
    return(core_node_g)
}

In [4]:
core_node_415 <- create_core_node_graph(id = 415,graph = fb_graph ) 

In [5]:
N_r = V(core_node_415)[degree(core_node_415) == 24]

In [6]:
N_r

+ 11/160 vertices, from 665ab83:
 [1]  31  53  75  90  93 102 118 133 134 136 137

In [7]:
print(length(N_r))

[1] 11


In [8]:
compute_common_neighbors <- function(node_i,node_j,graph)
{
    common_neighbors_i_j = intersection(neighbors(graph, node_i),neighbors(graph, node_j))
    return(length(common_neighbors_i_j))
}

In [9]:
compute_jaccard_measure <- function(node_i,node_j,graph)
{
    inter = intersection(neighbors(graph, node_i),neighbors(graph, node_j))
    uni = union(neighbors(graph, node_i),neighbors(graph, node_j))
    return(length(inter)/length(uni))
}

In [10]:
compute_ada_mic_r_measure <- function(node_i,node_j,graph)
{
    inter = intersection(neighbors(graph, node_i),neighbors(graph, node_j))
    sum = 0
    for (node_k in inter){
        term = 1/(log(length(neighbors(graph, node_k))))
        sum = sum + term
    }
    return(sum)
}

In [11]:
calculate_avg_acc <- function(core_node_g,measure_function){
    user_acc_list = c()
    n_iter = 10
    N_r = V(core_node_g)[degree(core_node_g) == 24]
    for(node_i in as_ids(N_r)){
    
        step_acc_list = c()
        for (iter in c(1:n_iter)){
        
            S_i = as_ids(neighbors(core_node_g, node_i))
            R_i = c() 
            tmp_graph = core_node_g
            
            for (node_neighbor in S_i){
                random_num = runif(1, 0, 1)
                if (random_num <= 0.25){
                
                    R_i = append(R_i, node_neighbor)
                    tmp_graph = delete_edges(tmp_graph, edge(node_neighbor, node_i))
                }
            }
            node_i_remaining_neighbors = cbind(setdiff(S_i, R_i),node_i)
            node_i_remaining_not_neighbors = setdiff(as_ids(V(tmp_graph)), node_i_remaining_neighbors)
            measure = c()
            for (node_j in node_i_remaining_not_neighbors){
                measure = c(measure, measure_function(node_i,node_j,tmp_graph))
            }
            best_idx = sort(measure,decreasing=TRUE, index.return=TRUE)$ix
            P_i = node_i_remaining_not_neighbors[best_idx[1:length(R_i)]]
            step_acc = length(intersect(P_i,R_i ))/length(R_i)
            if(!is.na(step_acc)){
                step_acc_list = cbind(step_acc_list,step_acc)
            }
        }
        user_acc = mean(step_acc_list)
        user_acc_list = cbind(user_acc_list,user_acc) 
        print(sprintf("User %d : Average accuracy of friend recommendation algorithm is %2.5f", node_i,user_acc))  
    }
    result = mean(user_acc_list)
    print(sprintf("Average accuracy of friend recommendation algorithm  is %2.5f",result))
    return(result)
}

In [12]:
result_common_neighbor = calculate_avg_acc(core_node_g = core_node_415,measure_function = compute_common_neighbors)

[1] "User 31 : Average accuracy of friend recommendation algorithm is 0.29905"
[1] "User 53 : Average accuracy of friend recommendation algorithm is 1.00000"
[1] "User 75 : Average accuracy of friend recommendation algorithm is 0.87143"
[1] "User 90 : Average accuracy of friend recommendation algorithm is 0.82476"
[1] "User 93 : Average accuracy of friend recommendation algorithm is 0.41988"
[1] "User 102 : Average accuracy of friend recommendation algorithm is 1.00000"
[1] "User 118 : Average accuracy of friend recommendation algorithm is 0.84167"
[1] "User 133 : Average accuracy of friend recommendation algorithm is 1.00000"
[1] "User 134 : Average accuracy of friend recommendation algorithm is 1.00000"
[1] "User 136 : Average accuracy of friend recommendation algorithm is 0.91893"
[1] "User 137 : Average accuracy of friend recommendation algorithm is 0.98571"
[1] "Average accuracy of friend recommendation algorithm  is 0.83286"


In [13]:
result_jaccard = calculate_avg_acc(core_node_g = core_node_415,measure_function = compute_jaccard_measure)

[1] "User 31 : Average accuracy of friend recommendation algorithm is 0.09592"
[1] "User 53 : Average accuracy of friend recommendation algorithm is 0.95893"
[1] "User 75 : Average accuracy of friend recommendation algorithm is 0.90071"
[1] "User 90 : Average accuracy of friend recommendation algorithm is 0.80881"
[1] "User 93 : Average accuracy of friend recommendation algorithm is 0.47036"
[1] "User 102 : Average accuracy of friend recommendation algorithm is 0.98571"
[1] "User 118 : Average accuracy of friend recommendation algorithm is 0.89389"
[1] "User 133 : Average accuracy of friend recommendation algorithm is 0.96333"
[1] "User 134 : Average accuracy of friend recommendation algorithm is 0.98333"
[1] "User 136 : Average accuracy of friend recommendation algorithm is 0.89655"
[1] "User 137 : Average accuracy of friend recommendation algorithm is 0.92500"
[1] "Average accuracy of friend recommendation algorithm  is 0.80750"


In [14]:
result_adamic_adar = calculate_avg_acc(core_node_g = core_node_415,measure_function = compute_ada_mic_r_measure)

[1] "User 31 : Average accuracy of friend recommendation algorithm is 0.30345"
[1] "User 53 : Average accuracy of friend recommendation algorithm is 1.00000"
[1] "User 75 : Average accuracy of friend recommendation algorithm is 0.88893"
[1] "User 90 : Average accuracy of friend recommendation algorithm is 0.82960"
[1] "User 93 : Average accuracy of friend recommendation algorithm is 0.43811"
[1] "User 102 : Average accuracy of friend recommendation algorithm is 1.00000"
[1] "User 118 : Average accuracy of friend recommendation algorithm is 0.83671"
[1] "User 133 : Average accuracy of friend recommendation algorithm is 1.00000"
[1] "User 134 : Average accuracy of friend recommendation algorithm is 1.00000"
[1] "User 136 : Average accuracy of friend recommendation algorithm is 0.87405"
[1] "User 137 : Average accuracy of friend recommendation algorithm is 0.96905"
[1] "Average accuracy of friend recommendation algorithm  is 0.83090"
